In [1]:
import torch
print(torch.cuda.is_available()) 

True


In [2]:
# coding:utf-8
# 导入必备的工具包
from langchain.prompts import PromptTemplate
from get_vector import *
from model import ChatGLM2

# 加载FAISS向量库
EMBEDDING_MODEL = './m3e-base'
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
db = FAISS.load_local(r'faiss/apiDocs', embeddings, allow_dangerous_deserialization=True)


def get_related_content(related_docs):
    related_content = []
    for doc in related_docs:
        related_content.append(doc.page_content.replace('\n\n', '\n'))
    return '\n'.join(related_content)


def define_prompt():
    question = '调用api，创建一个x=1,y=1,z=1的节点'
    docs = db.similarity_search(question, k=1)
    # print(f'docs-->{docs}')
    related_docs = get_related_content(docs)

    # 构建模板
    PROMPT_TEMPLATE = """
           基于以下已知信息，简洁和专业的来回答用户的问题。不允许在答案中添加编造成分。
           已知内容:
           {context}
           问题:
           {question}"""
    prompt = PromptTemplate(input_variables=["context", "question"],
                            template=PROMPT_TEMPLATE)

    my_prompt = prompt.format(context=related_docs, question=question)
    return my_prompt

C:\Users\65197\AppData\Local\Temp\ipykernel_2168\2105758863.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
C:\Users\65197\.conda\envs\elite\Lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\65197\.conda\envs\elite\Lib\site-packages\transformers\utils\generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` inst

In [4]:
llm = ChatGLM2()
llm.load_model(r'C:\Users\65197\Downloads\chatglm')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

C:\Users\65197\.conda\envs\elite\Lib\site-packages\transformers\modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_l

In [5]:
my_prompt = define_prompt()
result = llm(my_prompt)
print(f'result-->{result}')

C:\Users\65197\AppData\Local\Temp\ipykernel_2168\882704842.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm(my_prompt)
The dtype of attention mask (torch.int64) is not bool


result-->要调用API创建一个具有x、y和z值的节点，您可以使用以下代码：

```python
from qtmodel import Node

# 创建一个节点
node = Node(x=1, y=1, z=1)

# 将节点插入到数据库中
mdb.add_node(node)
```

请注意，您需要将x、y和z值替换为实际值，并使用适当的数据类型来定义节点。在示例中，我使用了Python的Node类，该类定义了节点的x、y和z属性。


In [7]:
my_prompt = '1+1等于几'
result = llm(my_prompt)
print(f'result-->{result}')

result-->1+1等于2。
